# Trabajo Práctico 1

In [1]:
import pandas as pd


In [2]:
data=pd.read_csv('./properatti.csv')

In [3]:
data.info(
)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121220 entries, 0 to 121219
Data columns (total 26 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Unnamed: 0                  121220 non-null  int64  
 1   operation                   121220 non-null  object 
 2   property_type               121220 non-null  object 
 3   place_name                  121197 non-null  object 
 4   place_with_parent_names     121220 non-null  object 
 5   country_name                121220 non-null  object 
 6   state_name                  121220 non-null  object 
 7   geonames_id                 102503 non-null  float64
 8   lat-lon                     69670 non-null   object 
 9   lat                         69670 non-null   float64
 10  lon                         69670 non-null   float64
 11  price                       100810 non-null  float64
 12  currency                    100809 non-null  object 
 13  price_aprox_lo

In [4]:
data.describe()

,Unnamed: 0,geonames_id,lat,lon,price,price_aprox_local_currency,price_aprox_usd,surface_total_in_m2,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses
count,121220.000000,1.025030e+05,69670.000000,69670.000000,1.008100e+05,1.008100e+05,1.008100e+05,81892.000000,101313.000000,68617.000000,8.765800e+04,7899.000000,47390.000000,1.426200e+04
mean,60609.500000,3.574442e+06,-34.626210,-59.266290,4.685259e+05,4.229397e+06,2.397006e+05,233.795328,133.050181,2160.086916,6.912216e+03,17.452336,3.080840,5.009234e+03
std,34993.344153,3.541306e+05,1.980936,2.299922,2.260101e+06,6.904714e+06,3.913239e+05,1782.222147,724.351479,2759.288621,2.837864e+04,120.243621,1.860773,1.204403e+05
min,0.000000,3.427208e+06,-54.823985,-75.678931,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.600000,1.510204e+00,1.000000,1.000000,1.000000e+00
25%,30304.750000,3.430234e+06,-34.669065,-58.727040,1.100000e+05,1.583309e+06,8.973388e+04,50.000000,45.000000,1218.181818,1.550000e+03,1.000000,2.000000,1.000000e+03
50%,60609.500000,3.433910e+06,-34.597985,-58.480128,1.850000e+05,2.558452e+06,1.450000e+05,84.000000,75.000000,1800.000000,2.213115e+03,3.000000,3.000000,2.000000e+03
75%,90914.250000,3.836668e+06,-34.441299,-58.395908,4.200000e+05,4.675792e+06,2.650000e+05,200.000000,150.000000,2486.411765,3.355549e+03,6.000000,4.000000,4.000000e+03
max,121219.000000,6.948895e+06,4.545843,-53.733330,6.500000e+08,8.212711e+08,4.654544e+07,200000.000000,187000.000000,206333.333333,4.000000e+06,3150.000000,32.000000,1.000150e+07


In [5]:
data.sample(5)

,Unnamed: 0,operation,property_type,place_name,place_with_parent_names,country_name,state_name,geonames_id,lat-lon,lat,...,surface_covered_in_m2,price_usd_per_m2,price_per_m2,floor,rooms,expenses,properati_url,description,title,image_thumbnail
71075,71075,sell,apartment,Banfield,|Argentina|Bs.As. G.B.A. Zona Sur|Lomas de Zam...,Argentina,Bs.As. G.B.A. Zona Sur,3436152.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,500.0,http://www.properati.com.ar/1akfy_venta_depart...,Hermoso departamento al frente a mts de la Est...,DEPTO A MTS DE LA EST. DE BANF.,https://thumbs4.properati.com/7/UiO35kWJTEv-3S...
16872,16872,sell,store,Centro,|Argentina|Buenos Aires Costa Atlántica|Mar de...,Argentina,Buenos Aires Costa Atlántica,3435548.0,"-37.9962919029,-57.5507433074",-37.996292,...,30.0,1166.666667,1166.666667,NaN,NaN,NaN,http://www.properati.com.ar/16zpi_venta_local_...,Local con entrepiso y stano en Galera Luro. Su...,"Local en Galera Luro, con entrepiso y stano",https://thumbs4.properati.com/0/exklCAt0RG3R99...
83788,83788,sell,house,Escobar,|Argentina|Bs.As. G.B.A. Zona Norte|Escobar|,Argentina,Bs.As. G.B.A. Zona Norte,3434130.0,NaN,NaN,...,220.0,1137.254902,1318.181818,NaN,NaN,8000.0,http://www.properati.com.ar/1b0yp_venta_casa_e...,"Casa en lote central, muy buena ubicación en e...",Casa - Miraflores,https://thumbs4.properati.com/5/HSNHM7QmCBcB9W...
41020,41020,sell,PH,Palermo,|Argentina|Capital Federal|Palermo|,Argentina,Capital Federal,3430234.0,"-34.58208,-58.42535",-34.582080,...,65.0,NaN,2907.692308,2.0,3.0,NaN,http://www.properati.com.ar/18t4n_venta_ph_pal...,Corredor Responsable: Maria Cristina Barrionue...,"Departamento tipo PH, 3amb 2do por escalera di...",https://thumbs4.properati.com/1/Ieid6x4lreUdhH...
113160,113160,sell,apartment,Bs.As. G.B.A. Zona Oeste,|Argentina|Bs.As. G.B.A. Zona Oeste|,Argentina,Bs.As. G.B.A. Zona Oeste,3435907.0,NaN,NaN,...,57.0,2006.470588,2393.684211,NaN,2.0,NaN,http://www.properati.com.ar/1c7zs_venta_depart...,VENTA DEPARTAMENTOS en CONDOMINIO TERRUAR | B°...,DEPARTAMENTO EN VENTA,https://thumbs4.properati.com/5/SxraF4t4vc7cdi...


 # Glosario columnas
 
-  price_aprox_usd:             
-  surface_total_in_m2:       
-  surface_covered_in_m2:      
-  price_usd_per_m2:           
-  price_per_m2:
-  floor:
-  rooms:
-  expenses:
-  properati_url:
-  description:
-  title:
-  image_thumbnail: